Install transformers if they aren't installed yet.

In [ ]:
!pip install transformers rouge-score nltk datasets
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 43.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM
from torchmetrics.text.rouge import ROUGEScore
import os
import pandas as pd
import numpy as np
import torch
import csv
import tqdm
# file_path = "/content/test.csv"
# df = (pd.read_csv(file_path, on_bad_lines='skip'))
rouges = []


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from datasets import DatasetDict, Dataset
import pandas as pd
# %%
# path for when you use uploaded files:
directory_path = os.getcwd()
# I uploaded all files in a folder content/data/*.csv:
directory_path = directory_path + "/data"

# path for files from drive:
directory_path = "/content/drive/MyDrive/MAG-1/NLP/IMP-corpus-csv-sentence"


# Read all .csv files in the directory into a list of pandas DataFrames
dfs = []
for file in os.listdir(directory_path):
    if file.endswith(".csv"):
        file_path = os.path.join(directory_path, file)
        df = pd.read_csv(file_path)
        dfs.append(df)

# Concatenate all DataFrames into one combined DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

# Convert the combined DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(combined_df)

# Split the dataset into train, validate, and test sets
# train_dataset = dataset.shuffle(seed=42).select(range(int(len(dataset) * 0.7)))
# valid_dataset = dataset.shuffle(seed=42).select(range(int(len(dataset) * 0.7), int(len(dataset) * 0.85)))
test_dataset = dataset.shuffle(seed=42).select(range(int(len(dataset) * 0.85), len(dataset)))

# Create a DatasetDict object with train, validate, and test sets
# dataset_dict = DatasetDict({
#     'train': train_dataset,
#     'validate': valid_dataset,
#     'test': test_dataset
# })

# Access the individual datasets using dictionary-like syntax
# train_dataset = dataset_dict['train']
# valid_dataset = dataset_dict['validate']
# test_dataset = dataset_dict['test']

Function translates accepts a sentance and translates it from modern to old slovene, it accepts following parameters:
- sentance: sentance to be translated

Other parameters are set to default but can be changed to see, if they improve performace. Others can be added as well. More about these parameters can be read on this link: https://huggingface.co/docs/transformers/main_classes/text_generation

In [ ]:
def translate(sentence, top_k=50, top_p=1, num_return_sequences=1):
  sentence = "translate: " + sentence + " </s>"
  encoding = tokenizer.encode_plus(sentence,padding=True, return_tensors="pt").to(device)
  input_ids, attention_masks = encoding["input_ids"], encoding["attention_mask"]
  outputs = model.generate(
      input_ids=input_ids, attention_mask=attention_masks,
      max_length=512,
      do_sample=True,
      top_k=top_k,
      top_p=top_p,
      early_stopping=True,
      num_return_sequences=num_return_sequences
  )
  # for i in range(len(outputs)):
  #   output = outputs[i]
  #   line = tokenizer.decode(output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
  #   if res is None:
  #     res = line
  #   print("Output " + str(i) + ": " + line)
  return tokenizer.decode(outputs[0], skip_special_tokens=True,clean_up_tokenization_spaces=True)

In [ ]:
def compare_strings(orig, reg, pred):
  origs = orig.split(" ")
  regs = reg.split(" ")
  preds = pred.split(" ")
  os = ""
  rs = ""
  ps = ""
  changed = 0
  all = 0
  for o, r, p in zip(origs, regs, preds):
    if o.lower() != r.lower():
      all += 1 
      os += o.lower() + " "
      rs += r.lower() + " "
      ps += p.lower() + " "
      if r.lower() != o.lower():
        changed += 1
  too_many = abs(len(preds) - len(origs))
  rougeOR = rouge(os, rs)
  rougePR = rouge(ps, os)
  if all == 0:
    changed = 0
  else:
    changed = changed/all
  return rougeOR, rougePR, changed, too_many


In [ ]:
def return_rouges(rouge_score):
  return rouge_score['rouge1_fmeasure'], rouge_score['rouge2_fmeasure'], rouge_score['rougeL_fmeasure'], rouge_score['rougeLsum_fmeasure']

In [ ]:
all_text_string = '''{0}
Orig: {1}
Pred: {2}
Regs: {3}
'''
diff_string = '''Full sentance {0}: {1:8.2f}
Chg. sentances {0}: {2:8.2f}
'''
all_string = '''\nNormal original text:\nr1: {0:8.2f} | r2: {1:8.2f} | rL: {2:8.2f} | rLsum: {3:8.2f}
Normal translated text:\nr1: {4:8.2f} | r2: {5:8.2f} | rL: {6:8.2f} | rLsum: {7:8.2f}
Changed original text:\nr1: {8:8.2f} | r2: {9:8.2f} | rL: {10:8.2f} | rLsum: {11:8.2f}
Changed translated text:\nr1: {12:8.2f} | r2: {13:8.2f} | rL: {14:8.2f} | rLsum: {15:8.2f} | changed: {16:8.2f} | too many words: {17}'''

In [ ]:
def write_file(file_name, file):
    with open('/content/drive/MyDrive/MAG-1/NLP/' + file_name, 'w', encoding="utf-8") as f:
      f.write(file)

In [ ]:
def load_model(model_link):
  global tokenizer 
  tokenizer = AutoTokenizer.from_pretrained(model_link)
  global model
  model = AutoModelForSeq2SeqLM.from_pretrained(model_link).to(device)

In [ ]:
def get_rouge_score(rouge_array):
  counter = 0
  diff = []
  for rouge_score in rouge_array:
    for r in return_rouges(rouge_score):
      rouges_scores_all[counter] += r
      if counter % 4 == 0:
        diff.append(r)
      counter += 1
  return (diff[1] - diff[0]), (diff[3] - diff[2])

In [ ]:
def evaluate_all_test(model_name, model_link, to_el, parameters):
  df = test_dataset[0:to_el]
  load_model(model_link)
  change = 0
  counter = len(df['orig'])
  all_texts =""
  difference = ""
  too_many = 0
  for i in tqdm.tqdm(range(counter)):
    reg = df['reg'][i]
    orig = df['orig'][i]
    if parameters:
      pred = translate(orig, top_k=100, top_p=0.60)
    else:
      pred = translate(orig)
    ro, rp, c, tm = compare_strings(orig, reg, pred)
    change+=c
    too_many += tm
    diff1, diff2 = get_rouge_score([rouge(orig, reg), rouge(orig, pred), ro, rp])
    all_texts += all_text_string.format(str(i), orig, pred, reg)
    difference += diff_string.format(str(i), float(diff1), float(diff2))
  rouge_scores_all = np.divide(rouges_scores_all, counter)
  all = model_name + all_string.format(rouge_scores_all[0], rouge_scores_all[1], rouge_scores_all[2], rouge_scores_all[3],
                                       rouge_scores_all[4], rouge_scores_all[5], rouge_scores_all[6], rouge_scores_all[7],
                                       rouge_scores_all[8], rouge_scores_all[9], rouge_scores_all[10], rouge_scores_all[11],
                                       rouge_scores_all[12], rouge_scores_all[13], rouge_scores_all[14], rouge_scores_all[15], change/counter, too_many)
  write_file(model_name + "_diff.txt", difference)
  write_file(model_name + "_texts.txt", all_texts)
  write_file(model_name + "_all.txt", all)

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
rouge = ROUGEScore()
rouges_scores_all = np.zeros(16)
model_repo_name = "martinjurkovic/mt5-base-finetuned-old-slovene"
model_name = "T5-small"
number_of_cases = 5000
evaluate_all_test(model_name, model_repo_name, number_of_cases, False)
rouges_scores_all = np.zeros(16)
evaluate_all_test(model_name + "-parameters", model_repo_name, number_of_cases, True)

100%|██████████| 5000/5000 [1:14:14<00:00,  1.12it/s]
